task3学习内容

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib as mpl
from sklearn.model_selection import train_test_split

In [2]:
#正常显示中文
mpl.rcParams['font.sans-serif']=[u'SimHei']
mpl.rcParams['axes.unicode_minus'] = False
#用于抑制第三方警告
warnings.filterwarnings("ignore")

In [63]:
pd.options.display.max_rows = 10   #最大显示10行
data = pd.read_csv(r"D:\datasets\data.csv",encoding = "gbk")  #读取数据  

data.info()  #显示数据的详细信息，有没有空行等。

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4754 entries, 0 to 4753
Data columns (total 90 columns):
Unnamed: 0                                    4754 non-null int64
custid                                        4754 non-null int64
trade_no                                      4754 non-null object
bank_card_no                                  4754 non-null object
low_volume_percent                            4752 non-null float64
middle_volume_percent                         4752 non-null float64
take_amount_in_later_12_month_highest         4754 non-null int64
trans_amount_increase_rate_lately             4751 non-null float64
trans_activity_month                          4752 non-null float64
trans_activity_day                            4752 non-null float64
transd_mcc                                    4752 non-null float64
trans_days_interval_filter                    4746 non-null float64
trans_days_interval                           4752 non-null float64
regional_mobility

In [106]:
data_1 = data.copy()
data_1.describe()

,Unnamed: 0,custid,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
count,4754.000000,4.754000e+03,4752.000000,4752.000000,4754.000000,4751.000000,4752.000000,4752.000000,4752.000000,4746.000000,...,4457.000000,4457.000000,4457.000000,4457.000000,4457.000000,4457.000000,4457.000000,4457.000000,4450.000000,4457.000000
mean,6008.414178,1.690993e+06,0.021806,0.901294,1940.197728,14.160674,0.804411,0.365425,17.502946,29.029920,...,3390.038142,1820.357864,9187.009199,76.042630,4.732331,5.227507,16153.690823,8007.696881,24.112809,55.181512
std,3452.071428,1.034235e+06,0.041527,0.144856,3923.971494,694.180473,0.196920,0.170196,4.475616,22.722432,...,1474.206546,583.418291,7371.257043,14.536819,2.974596,3.409292,14301.037628,5679.418585,37.725724,53.486408
min,5.000000,1.140000e+02,0.000000,0.000000,0.000000,0.000000,0.120000,0.033000,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000,-2.000000
25%,3106.000000,7.593358e+05,0.010000,0.880000,0.000000,0.615000,0.670000,0.233000,15.000000,16.000000,...,2300.000000,1535.000000,4800.000000,77.000000,2.000000,3.000000,7800.000000,4737.000000,5.000000,10.000000
50%,6006.500000,1.634942e+06,0.010000,0.960000,500.000000,0.970000,0.860000,0.350000,17.000000,23.000000,...,3100.000000,1810.000000,7700.000000,79.000000,4.000000,5.000000,13800.000000,7050.000000,14.000000,36.000000
75%,8999.000000,2.597905e+06,0.020000,0.990000,2000.000000,1.600000,1.000000,0.480000,20.000000,32.000000,...,4300.000000,2100.000000,11700.000000,80.000000,7.000000,7.000000,20400.000000,10000.000000,24.000000,91.000000
max,11992.000000,4.004694e+06,1.000000,1.000000,68000.000000,47596.740000,1.000000,0.941000,42.000000,285.000000,...,10000.000000,6900.000000,87100.000000,87.000000,18.000000,20.000000,266400.000000,82800.000000,360.000000,323.000000


In [107]:
#将空值替换为np.nan
for columns in data_1.columns:
    data_1.loc[data_1[columns] == "NA",columns] = np.nan  

In [108]:
#删除无效特征
Irrelevant_features = ['Unnamed: 0', 'custid', 'trade_no', 'bank_card_no', 'source', 'id_name', 'latest_query_time','loans_latest_time']
data_2 = data_1.drop(Irrelevant_features,axis = 1)

In [118]:
#根据reg_preference_for_trad删除空值
data_3 = data_2.dropna(subset = ["reg_preference_for_trad"])
data_3 = data_2.dropna(subset = ["status"])

#在指定阈值的前提下对特征进行删除 
data_3 = data_3.dropna(axis = 1,thresh = 1000)
data_3 = data_3.dropna(axis = 0,thresh = 75)

#对空值采用均值进行填充
data_3 = data_3.fillna(data_3.mean())
print(data_3.shape)

(4423, 82)


In [131]:
#去除标签
data_non_labels = data_3.drop("status",axis = 1)

#采样标签
data_labels = data_3["status"]

#切分数据集
X_train_set,X_test_set,y_train_set,y_test_set = train_test_split(data_non_labels,data_labels,test_size = 0.3,random_state = 2018)
print(X_train_set.shape)

(3096, 81)


In [132]:
#取reg_preference_for_trad
reg_preference_for_trad = X_train_set["reg_preference_for_trad"].copy()
print(reg_preference_for_trad.shape)

#去除reg_preference_for_trad
data_non_reg_preference = X_train_set.drop("reg_preference_for_trad",axis = 1)
print(data_non_reg_preference.shape)

(3096,)
(3096, 80)


In [133]:
#对数据进行归一化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_non_reg_preference_scalered = scaler.fit_transform(data_non_reg_preference)
print(data_non_reg_preference_scalered.shape)

(3096, 80)


In [134]:
#尝试着将城市信息转变为onehot矩阵  
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
encoder = LabelEncoder().fit_transform(reg_preference_for_trad)
reg_preference_encodered =  OneHotEncoder().fit_transform(encoder.reshape(-1,1)).toarray()
reg_preference_encodered.shape

(3096, 5)

In [135]:
#将reg_preference_for_trad再添加进去
data_prepared = np.hstack((data_non_reg_preference_scalered,reg_preference_encodered))
print(data_prepared.shape)

(3096, 85)


In [58]:
#方差选择法
from sklearn.feature_selection import VarianceThreshold
t = 4
vt = VarianceThreshold(threshold = t)
vt.fit_transform(data_non_labels.drop("reg_preference_for_trad",axis = 1))
vt.variances_

array([1.45919250e-03, 1.97904426e-02, 1.60462249e+07, 5.17605382e+05,
       3.86711522e-02, 2.88755421e-02, 1.97168252e+01, 5.12145759e+02,
       2.63740768e+02, 7.91795156e-01, 4.51735811e-04, 2.80612036e+09,
       1.09581712e-02, 1.00365486e+02, 2.20011034e+08, 1.06120895e+11,
       9.93578273e+03, 6.87048148e-02, 1.07611512e+10, 1.85518565e+00,
       7.58363709e+08, 1.21421986e-01, 5.64464541e+05, 9.67012598e-03,
       2.82557203e+01, 1.65534318e+02, 2.98323385e+01, 1.72191896e+02,
       5.27046596e+00, 3.68985909e+00, 2.04259183e+01, 2.29850672e+01,
       1.35904361e+11, 1.24086087e+08, 3.30175008e+01, 2.44106055e-01,
       4.58571601e+07, 8.09862543e+08, 4.34156851e-02, 2.89780224e+05,
       3.31715233e+02, 2.60383185e+03, 1.72430879e+01, 4.95058104e+01,
       1.44804408e+01, 6.74871664e+00, 1.27673048e+02, 2.05241508e+01,
       5.81930539e+01, 8.60936312e+01, 3.71488743e+03, 4.95104394e+00,
       6.04181517e+02, 4.69177596e+02, 9.95202624e+00, 5.51748752e+01,
      

In [153]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
data_labels = y_train_set
lin_reg.fit(data_prepared,data_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [167]:
#线性回归欠拟合
from sklearn.metrics import mean_squared_error  #采用均方误差进行评价
some_data = data_prepared[:5,:]
some_data_labels = data_labels[:5]
lin_predictions = lin_reg.predict(some_data)

lin_mse = mean_squared_error(some_data_labels,lin_predictions)
print(lin_mse)

0.3094568069334855


In [170]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(data_prepared,data_labels)
log_predictions = log_reg.predict(some_data)

In [171]:
log_mse = mean_squared_error(some_data_labels,log_predictions)
print(log_mse)

0.4


In [172]:
#决策树过拟合  
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(data_prepared,data_labels)
tree_predictions = tree_reg.predict(some_data)
tree_mse = mean_squared_error(some_data_labels,tree_predictions)
print(tree_mse)

0.0


In [173]:
#  随机森林  
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(data_prepared,data_labels)
forest_reg_predictions = forest_reg.predict(some_data)
forest_reg_mse = mean_squared_error(some_data_labels,forest_reg_predictions)
print(forest_reg_mse)

0.06000000000000001


In [176]:
#应用SVC
from sklearn.svm import SVC
svc_reg = SVC()
svc_reg.fit(data_prepared,data_labels)
svc_predictions = svc_reg.predict(some_data)
svc_mse = mean_squared_error(some_data_labels,svc_predictions)
print(svc_mse)

0.4
